## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
number_of_success = application_df[application_df['IS_SUCCESSFUL']==1]['IS_SUCCESSFUL'].count()
number_of_failure = application_df.shape[0] - number_of_success
print(f"Out of {application_df.shape[0]} records, {number_of_success} or {round((number_of_success/application_df.shape[0])*100,2)}% were successfull and {number_of_failure} or {round((number_of_failure/application_df.shape[0])*100,2)}% were not! This means that the data is well balanced between efficient and inefficient organizations.") 

Out of 34299 records, 18261 or 53.24% were successfull and 16038 or 46.76% were not! This means that the data is well balanced between efficient and inefficient organizations.


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [6]:
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [7]:
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df.groupby('APPLICATION_TYPE').filter(lambda x: len(x) < 20)['APPLICATION_TYPE']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       27
T12         27
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# application_d_df.groupby('CLASSIFICATION').filter(lambda x: len(x) > 1)['CLASSIFICATION'].value_counts()
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()>10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df.groupby('CLASSIFICATION').filter(lambda x: len(x) < 50)['CLASSIFICATION']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
X = pd.get_dummies(application_df)
X.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
y = X['IS_SUCCESSFUL']
X = X.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
X_train_scaled.shape

(25724, 56)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 30
# hidden_nodes_layer4 = 10
# hidden_nodes_layer5 = 5

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
# nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4560      
                                                                 
 dense_1 (Dense)             (None, 40)                3240      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 7841 (30.63 KB)
Trainable params: 7841 (30.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', save_weights_only=True, period=5)
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 980us/step - loss: 0.5692 - accuracy: 0.7225
Epoch 2/200
804/804 [==============================] - 1s 951us/step - loss: 0.5539 - accuracy: 0.7311
Epoch 3/200
804/804 [==============================] - 1s 956us/step - loss: 0.5504 - accuracy: 0.7314
Epoch 4/200
804/804 [==============================] - 1s 952us/step - loss: 0.5492 - accuracy: 0.7326
Epoch 5/200
804/804 [==============================] - 1s 956us/step - loss: 0.5474 - accuracy: 0.7336
Epoch 6/200
804/804 [==============================] - 1s 959us/step - loss: 0.5461 - accuracy: 0.7346
Epoch 7/200
804/804 [==============================] - 1s 975us/step - loss: 0.5448 - accuracy: 0.7350
Epoch 8/200
804/804 [==============================] - 1s 956us/step - loss: 0.5446 - accuracy: 0.7353
Epoch 9/200
804/804 [==============================] - 1s 950us/step - loss: 0.5437 - accuracy: 0.7345
Epoch 10/200
804/804 [==============================] - 1s 962us/step - l

804/804 [==============================] - 1s 975us/step - loss: 0.5308 - accuracy: 0.7425
Epoch 79/200
804/804 [==============================] - 1s 959us/step - loss: 0.5314 - accuracy: 0.7422
Epoch 80/200
804/804 [==============================] - 1s 962us/step - loss: 0.5311 - accuracy: 0.7427
Epoch 81/200
804/804 [==============================] - 1s 965us/step - loss: 0.5309 - accuracy: 0.7432
Epoch 82/200
804/804 [==============================] - 1s 972us/step - loss: 0.5309 - accuracy: 0.7438
Epoch 83/200
804/804 [==============================] - 1s 965us/step - loss: 0.5304 - accuracy: 0.7426
Epoch 84/200
804/804 [==============================] - 1s 987us/step - loss: 0.5311 - accuracy: 0.7423
Epoch 85/200
804/804 [==============================] - 1s 987us/step - loss: 0.5307 - accuracy: 0.7432
Epoch 86/200
804/804 [==============================] - 1s 966us/step - loss: 0.5312 - accuracy: 0.7430
Epoch 87/200
804/804 [==============================] - 1s 965us/step - loss:

804/804 [==============================] - 1s 969us/step - loss: 0.5284 - accuracy: 0.7442
Epoch 157/200
804/804 [==============================] - 1s 954us/step - loss: 0.5284 - accuracy: 0.7441
Epoch 158/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5286 - accuracy: 0.7437
Epoch 159/200
804/804 [==============================] - 1s 975us/step - loss: 0.5282 - accuracy: 0.7434
Epoch 160/200
804/804 [==============================] - 1s 974us/step - loss: 0.5333 - accuracy: 0.7437
Epoch 161/200
804/804 [==============================] - 1s 999us/step - loss: 0.5286 - accuracy: 0.7431
Epoch 162/200
804/804 [==============================] - 1s 960us/step - loss: 0.5285 - accuracy: 0.7443
Epoch 163/200
804/804 [==============================] - 1s 992us/step - loss: 0.5282 - accuracy: 0.7440
Epoch 164/200
804/804 [==============================] - 1s 951us/step - loss: 0.5283 - accuracy: 0.7435
Epoch 165/200
804/804 [==============================] - 1s 954us/step 

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5635 - accuracy: 0.7277 - 292ms/epoch - 1ms/step
Loss: 0.5634929537773132, Accuracy: 0.7276967763900757


In [21]:
# Export our model to HDF5 file
# import os
nn_model.save_weights('models/AlphabetSoupCharity_Optimization_model_3.hdf5')
# save_model('models/model_1.json')